In [1]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import make_column_transformer

решил взять этот датасет https://www.kaggle.com/colearninglounge/predicting-pulsar-starintermediate
В нем собраны данные о звездах и предстоит предсказать превратиться ли звезда в пульсар или нет. Поскольку пульсары достаточно редкие, то данные существенно разрежены.
При этом сами данные прдеставляют собой статистические оценки распределения электромагнитных импульсов нейтронных звезд.
Первые четыре фичи - это оценки приходящего на змелю импульса от конкртеного объекта, причем насколько я понял испульсы от конкретно пульсаров приходят не с одной частотой и имеют разную форму и могут иметь нехарактерные выбросы. Поэтому астрономы собирают по конкретной звезде сигнатуры импульсов и усредняют их и на выходе получаются 4 первые фичи которые есть в датасете.  
  
А остальные 4 - оценки DM-SNR curve....насколько я понял это связано тоже с импульсом, только с тем, что импульс приходит с  шумом и на разных частотах, связанно это с ионизаией межзвездного пространства и называется в англоязычных источниках "dispersion" - не знаю как наши астрономы это называют, но переводить как дисперсию не совсем корректно как мне кажется. В общем каким-то образом астрономы считают для каждого приходящего сигнала от звезды "dispersion-measure-signal-to-noise-ratio" и так же их усредняют. Вот собственно последние 4 фичи как раз и есть статистические оценки этих кривых для каждой звезды.

In [2]:
df = pd.read_csv("./data/HW_6/pulsar_data_train.csv")

In [3]:
df.dtypes

 Mean of the integrated profile                  float64
 Standard deviation of the integrated profile    float64
 Excess kurtosis of the integrated profile       float64
 Skewness of the integrated profile              float64
 Mean of the DM-SNR curve                        float64
 Standard deviation of the DM-SNR curve          float64
 Excess kurtosis of the DM-SNR curve             float64
 Skewness of the DM-SNR curve                    float64
target_class                                     float64
dtype: object

In [4]:
df.head(3)

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class
0,121.156250,48.372971,0.375485,-0.013165,3.168896,18.399367,7.449874,65.159298,0.0
1,76.968750,36.175557,0.712898,3.388719,2.399666,17.570997,9.414652,102.722975,0.0
2,130.585938,53.229534,0.133408,-0.297242,2.743311,22.362553,8.508364,74.031324,0.0


In [5]:
df.shape

(12528, 9)

In [6]:
#  примеров много, это хорошо. Посмотрим на баланс классов

In [7]:
df["target_class"].value_counts()

0.0    11375
1.0     1153
Name: target_class, dtype: int64

In [8]:
#  Видим существенный дисбаланс, посчти в 10 раз. Попробуем решить задачу PU-learning, но сначала попробуем пофитить данные.
#  Честно признаться что делать с такими данными ума не приложу. Но раз они все непрервные и иих мало, то предлагаю 
#  Для начала отмасштабировать их, а затем к каждой применить преобразование в **2 и логарифмическое преобразование

In [9]:
#  посмотрим сколько пропущенных значений

In [10]:
df.isna().sum()

 Mean of the integrated profile                     0
 Standard deviation of the integrated profile       0
 Excess kurtosis of the integrated profile       1735
 Skewness of the integrated profile                 0
 Mean of the DM-SNR curve                           0
 Standard deviation of the DM-SNR curve          1178
 Excess kurtosis of the DM-SNR curve                0
 Skewness of the DM-SNR curve                     625
target_class                                        0
dtype: int64

In [11]:
#  ну в целом встречаются, но их немного....сделаем пайплайн, который будет заменять пропущенные значения на медиану

In [12]:
from sklearn.model_selection import train_test_split

In [13]:
#  разделим данные
X_train, X_test, y_train, y_test = train_test_split(df.loc[:, :" Skewness of the DM-SNR curve"], df[["target_class"]], 
                                                    test_size=0.3, random_state=42)

In [14]:
class ReplaceNan(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.median_dict = {}

    def fit(self, X, y=None):
        for col in X.columns:
            self.median_dict[col] = X[col].median()
        return self

    def transform(self, X):
        out = X.copy()
        for col in X.columns:
            out.loc[out[col].isna(), col] = self.median_dict[col]
        return out

In [15]:
class CreateDf(BaseEstimator, TransformerMixin):
    def __init__(self, col_names):
        self.col_names = col_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = X.copy()
        return pd.DataFrame(out, columns=self.col_names)

In [16]:
class PolyFeatureAll(BaseEstimator, TransformerMixin):

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        out = X.copy()
        for col in out.columns:
            out[f"{col}_squared"] = out[col] ** 2
            out[f"{col}_log"] = np.where(out[f"{col}_squared"] == 0, 
                                         0, 
                                         np.log(out[f"{col}_squared"])
                                        )
        return out

#  Поскольку я собираюьс использовать StandartScaler, а там могут присутсвовать отрицательные значения, что для логарифма 
#  неприемлимо, то в качестве аргумента логарифма я буду передавать квадрат соответсвующей фичи
#  полюс учесть вариант с нулем, потому что логарифм нуля не определен

In [17]:
scaler = make_column_transformer(
                                 (StandardScaler(), 
                                  [' Mean of the integrated profile', ' Standard deviation of the integrated profile',
                                   ' Excess kurtosis of the integrated profile', ' Skewness of the integrated profile', 
                                   ' Mean of the DM-SNR curve', ' Standard deviation of the DM-SNR curve', 
                                   ' Excess kurtosis of the DM-SNR curve', ' Skewness of the DM-SNR curve'])
                                 )

In [18]:
feature_engine = make_pipeline(scaler, 
                               CreateDf(["mean_of_intprof", "std_of_intprof", "kurtosis_of_intprof", "skewness_of_intprof",
                                         "mean_of_dmsnr", "std_of_dmsnr", "kurtosis_of_dmsnr", "skewness_of_dmsnr"]),
                               ReplaceNan())

In [19]:
feature_engine.fit_transform(X_train).describe()

,mean_of_intprof,std_of_intprof,kurtosis_of_intprof,skewness_of_intprof,mean_of_dmsnr,std_of_dmsnr,kurtosis_of_dmsnr,skewness_of_dmsnr
count,8.769000e+03,8.769000e+03,8769.000000,8.769000e+03,8.769000e+03,8769.000000,8.769000e+03,8769.000000
mean,1.917474e-16,-1.733513e-16,-0.033240,-9.422779e-18,3.089228e-18,-0.037475,-2.269063e-16,-0.010248
std,1.000057e+00,1.000057e+00,0.931692,1.000057e+00,1.000057e+00,0.959965,1.000057e+00,0.975907
min,-4.093953e+00,-3.146608e+00,-2.069390,-5.732479e-01,-4.209666e-01,-0.975739,-2.466672e+00,-1.003284
25%,-4.006069e-01,-6.092257e-01,-0.395392,-3.161286e-01,-3.634431e-01,-0.595377,-5.584327e-01,-0.631312
50%,1.573829e-01,6.148837e-02,-0.239312,-2.549724e-01,-3.334553e-01,-0.405699,2.869776e-02,-0.206107
75%,6.301579e-01,6.585041e-01,-0.054473,-1.362838e-01,-2.435483e-01,0.024536,5.326459e-01,0.288401
max,3.045283e+00,6.590582e+00,7.099863,1.066318e+01,6.626952e+00,4.232422,5.827287e+00,10.239105


In [20]:
feature_engine.fit_transform(X_train).isna().sum()

mean_of_intprof        0
std_of_intprof         0
kurtosis_of_intprof    0
skewness_of_intprof    0
mean_of_dmsnr          0
std_of_dmsnr           0
kurtosis_of_dmsnr      0
skewness_of_dmsnr      0
dtype: int64

In [21]:
# посмотрим как отработала замена пустых значений
# мы не можем посмотртеь это по индексам, потому что в тренировочном сете они не по порядку, а в обработанном они как новый
# датафрейм создаются. 
# Но мы можем посчитать количество пустых и посчитать количество медианной оценки в обработанном фреме и посмотреть 

In [22]:
# в классе по замене пропущенных значений есть параметр обучения, до которого можно достучаться
median_dict = feature_engine.get_params()["steps"][-1][-1].median_dict
median_dict

{'mean_of_intprof': 0.15738290226572363,
 'std_of_intprof': 0.06148837442974762,
 'kurtosis_of_intprof': -0.23931228446966626,
 'skewness_of_intprof': -0.25497243920583584,
 'mean_of_dmsnr': -0.33345529633325094,
 'std_of_dmsnr': -0.4056989073055492,
 'kurtosis_of_dmsnr': 0.028697762809245207,
 'skewness_of_dmsnr': -0.20610736699842797}

In [23]:
X_train.loc[X_train[" Standard deviation of the DM-SNR curve"].isna()].shape[0]
#  пропущенные значения по 6 фиче

810

In [24]:
(feature_engine.fit_transform(X_train)["std_of_dmsnr"] == median_dict["std_of_dmsnr"]).sum()

811

In [25]:
#  811 наверно медиана попала на конкретное значение, поэтому в обработанном фрейме единицу больше

In [26]:
X_train.shape[0]

8769

In [27]:
#  ну так и есть, наблюдений неченое число

In [28]:
#  посмотрим на всякий случай по другой фиче

In [29]:
X_train.loc[X_train[" Excess kurtosis of the integrated profile"].isna()].shape[0]
#  пропущенные значения по 3 фиче

1218

In [30]:
(feature_engine.fit_transform(X_train)["kurtosis_of_intprof"] == median_dict["kurtosis_of_intprof"]).sum()

1219

In [31]:
#  отлично, все отрабатывает вроде всерно
#  перезапишем трансформер добавив степенное преобразование

In [32]:
feature_engine = make_pipeline(scaler, 
                               CreateDf(["mean_of_intprof", "std_of_intprof", "kurtosis_of_intprof", "skewness_of_intprof",
                                         "mean_of_dmsnr", "std_of_dmsnr", "kurtosis_of_dmsnr", "skewness_of_dmsnr"]),
                               ReplaceNan(),
                               PolyFeatureAll()
                              )

In [33]:
feature_engine.fit_transform(X_train)

,mean_of_intprof,std_of_intprof,kurtosis_of_intprof,skewness_of_intprof,mean_of_dmsnr,std_of_dmsnr,kurtosis_of_dmsnr,skewness_of_dmsnr,mean_of_intprof_squared,mean_of_intprof_log,...,skewness_of_intprof_squared,skewness_of_intprof_log,mean_of_dmsnr_squared,mean_of_dmsnr_log,std_of_dmsnr_squared,std_of_dmsnr_log,kurtosis_of_dmsnr_squared,kurtosis_of_dmsnr_log,skewness_of_dmsnr_squared,skewness_of_dmsnr_log
0,0.181037,0.218106,-0.294831,-0.320942,-0.263841,0.044253,-0.426087,-0.582606,0.032774,-3.418110,...,0.103004,-2.272987,0.069612,-2.664819,0.001958,-6.235653,0.181550,-1.706223,0.339430,-1.080488
1,-0.080673,0.674244,-0.238607,-0.276213,-0.358370,-0.593524,0.434472,0.239128,0.006508,-5.034715,...,0.076293,-2.573169,0.128429,-2.052379,0.352271,-1.043356,0.188766,-1.667249,0.057182,-2.861514
2,0.961009,-0.219836,-0.479055,-0.206964,-0.251355,0.132838,-0.428847,-0.591623,0.923539,-0.079542,...,0.042834,-3.150418,0.063179,-2.761776,0.017646,-4.037255,0.183910,-1.693311,0.350018,-1.049770
3,0.482473,0.421053,-0.417937,-0.329279,-0.289601,-0.121914,-0.277934,-0.475524,0.232780,-1.457662,...,0.108425,-2.221701,0.083869,-2.478502,0.014863,-4.208875,0.077247,-2.560744,0.226123,-1.486675
4,1.004375,-0.721643,-0.418720,-0.185203,-0.375168,-0.604979,0.822467,0.512726,1.008769,0.008731,...,0.034300,-3.372603,0.140751,-1.960765,0.366000,-1.005122,0.676452,-0.390893,0.262888,-1.336027
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8764,-0.211982,-2.030838,-0.012646,0.205623,-0.347604,-0.342031,0.174245,-0.158935,0.044936,-3.102507,...,0.042281,-3.163424,0.120828,-2.113385,0.116985,-2.145706,0.030361,-3.494587,0.025260,-3.678520
8765,-0.071878,-0.302066,-0.037593,-0.173809,-0.403042,-0.405699,2.104268,2.279705,0.005166,-5.265567,...,0.030209,-3.499602,0.162443,-1.817431,0.164592,-1.804288,4.427942,1.487935,5.197055,1.648092
8766,-3.896534,-1.677625,4.761961,4.792705,2.076080,1.492015,-1.613207,-0.975166,15.182978,2.720175,...,22.970018,3.134190,4.310109,1.460963,2.226110,0.800256,2.602438,0.956449,0.950949,-0.050295
8767,-0.213498,0.222297,-0.239312,-0.220054,-0.308202,-0.283934,-0.229892,-0.413254,0.045582,-3.088252,...,0.048424,-3.027761,0.094989,-2.353997,0.080619,-2.518026,0.052850,-2.940293,0.170779,-1.767388


In [34]:
#  ради интереса обучу быстренько модель на решающих деревьях чтобы посмотреть насколько хорошо классический подход работает

In [35]:
from sklearn.ensemble import RandomForestClassifier

In [36]:
forest_clf = RandomForestClassifier(n_estimators=500, n_jobs=-1, max_depth=7, criterion="entropy")

In [37]:
forest_pipe = make_pipeline(feature_engine, forest_clf)

In [38]:
forest_pipe.fit(X_train, y_train.to_numpy().reshape(-1))

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(transformers=[('standardscaler',
                                                                   StandardScaler(),
                                                                   [' Mean of '
                                                                    'the '
                                                                    'integrated '
                                                                    'profile',
                                                                    ' Standard '
                                                                    'deviation '
                                                                    'of the '
                                                                    'integrated '
                                                                    'profile',
                             

In [39]:
forest_clf_pred = forest_pipe.predict_proba(X_test)[:, 1]
forest_clf_pred

array([0.00669327, 0.00184825, 0.00360997, ..., 0.00617197, 0.01746299,
       0.39564519])

In [40]:
from sklearn.metrics import roc_auc_score

In [41]:
roc_score = roc_auc_score(y_test, forest_clf_pred)
roc_score

0.9792958268943823

In [42]:
#  хм.....а судя по всему получился неплохой классификатор....но надо посмотреть на confusion matrix... возможно 
#  из-за дисбаланса классов у нас такая хорошая оценка выходит

In [43]:
from sklearn.metrics import precision_recall_curve, confusion_matrix

In [44]:
precision, recall, thresholds = precision_recall_curve(y_test, forest_clf_pred)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=0.388516, F-Score=0.893, Precision=0.943, Recall=0.847


In [45]:
confusion_matrix(y_test, forest_clf_pred>thresholds[ix])

array([[3388,   18],
       [  55,  298]], dtype=int64)

In [46]:
#  ну в целом действительно неплохой классификатор получился. Мы с уверенностью можем вычленить все пульсары из общего массива
#  при этом мы очень мало делаем ошибок второго рода, т.е. FP. Думаю для целей разведочного научного исследования это лучше,
#  поскольку не нужно тратить ресурсы на то чтобы вычленить FP примеры из предсказанного массива....как если бы precision Был
#  бы на уровне 50-60%

In [47]:
#  запишем данные во фрейм
model_comparison = pd.DataFrame(np.array([thresholds[ix], precision[ix], recall[ix], fscore[ix], roc_score]).reshape(1, -1), 
                                columns=["Threshold", "Precision", "Recall", "F1_score", "ROC_AUC"], 
                                index=["Random_forest"])
model_comparison

,Threshold,Precision,Recall,F1_score,ROC_AUC
Random_forest,0.388516,0.943218,0.847025,0.892537,0.979296


In [48]:
#  сделаем функцию, которая будет возвращать фрем для обучения
def prepare_df_for_pulearner(df, percentage_of_positive, target_col_name):
    pu_df = df.copy()
    indexes_of_pos_class = pu_df.loc[pu_df[target_col_name] == 1, :].index
    indexes_of_neg_class = pu_df.loc[pu_df[target_col_name] == 0, :].index
    
    num_of_positive_class = len(indexes_of_pos_class)
    sample_length = int(num_of_positive_class * percentage_of_positive)
    pos_sample_index = np.random.choice(indexes_of_pos_class, sample_length, replace=False)
    neg_sample_index = np.random.choice(indexes_of_neg_class, sample_length, replace=False)
    
    pos_sample_df = pu_df.loc[pos_sample_index, :]
    pos_sample_df["pu_target_class"] = 1
    
    neg_sample_df = pu_df.loc[neg_sample_index, :]
    neg_sample_df["pu_target_class"] = -1
    
    out_df = pos_sample_df.append(neg_sample_df)
    out_df = out_df.sample(frac=1)
    
    print(f"Used {sample_length} instances of {num_of_positive_class} total positive class")
    return out_df
    

In [49]:
pu50_train = prepare_df_for_pulearner(df, 0.5, "target_class")

Used 576 instances of 1153 total positive class


In [50]:
pu30_test = prepare_df_for_pulearner(df, 0.3, "target_class")

Used 345 instances of 1153 total positive class


In [51]:
pu50_train.head(3)

,Mean of the integrated profile,Standard deviation of the integrated profile,Excess kurtosis of the integrated profile,Skewness of the integrated profile,Mean of the DM-SNR curve,Standard deviation of the DM-SNR curve,Excess kurtosis of the DM-SNR curve,Skewness of the DM-SNR curve,target_class,pu_target_class
11412,91.906250,49.618614,0.563407,0.394458,2.314381,16.086144,10.011777,114.664815,0.0,-1
12471,133.164062,52.981981,-0.132384,-0.493945,27.844482,NaN,2.141015,2.898546,0.0,-1
3870,43.000000,28.466537,4.154733,25.624947,25.643813,57.615931,2.477723,5.393948,1.0,1


In [52]:
pu50_X_train = pu50_train.iloc[:, :-2]
pu50_y_train = pu50_train.iloc[:, -1]

In [53]:
pu30_X_test = pu30_test.iloc[:, :-2]
pu30_y_test = pu30_test.iloc[:, -1]

In [54]:
#  сделаем пайплайн с тем же классификатором как и на базовой модели

In [55]:
forest_pipe_for_PU = make_pipeline(feature_engine, forest_clf)

In [56]:
forest_pipe_for_PU.fit(pu50_X_train, pu50_y_train)

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('columntransformer',
                                  ColumnTransformer(transformers=[('standardscaler',
                                                                   StandardScaler(),
                                                                   [' Mean of '
                                                                    'the '
                                                                    'integrated '
                                                                    'profile',
                                                                    ' Standard '
                                                                    'deviation '
                                                                    'of the '
                                                                    'integrated '
                                                                    'profile',
                             

In [57]:
forestPU_predict = forest_pipe_for_PU.predict(pu30_X_test)

In [58]:
roc_score = roc_auc_score(pu30_y_test, forestPU_predict)
roc_score

0.946376811594203

In [59]:
precision, recall, thresholds = precision_recall_curve(pu30_y_test, forestPU_predict)

fscore = (2 * precision * recall) / (precision + recall)
# locate the index of the largest f score
ix = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[ix], 
                                                                        fscore[ix],
                                                                        precision[ix],
                                                                        recall[ix]))

Best Threshold=1.000000, F-Score=0.945, Precision=0.978, Recall=0.913


In [60]:
pu_model = pd.DataFrame(np.array([thresholds[ix], precision[ix], recall[ix], fscore[ix], roc_score]).reshape(1, -1), 
                                columns=["Threshold", "Precision", "Recall", "F1_score", "ROC_AUC"], 
                                index=["Forest_PU_learne"])
pu_model

,Threshold,Precision,Recall,F1_score,ROC_AUC
Forest_PU_learne,1.0,0.978261,0.913043,0.944528,0.946377


In [61]:
model_comparison.append(pu_model)

,Threshold,Precision,Recall,F1_score,ROC_AUC
Random_forest,0.388516,0.943218,0.847025,0.892537,0.979296
Forest_PU_learne,1.000000,0.978261,0.913043,0.944528,0.946377


In [62]:
#  если смотреть чисто на roc_auc, то модель ожидаемо работает хуже, потому что обучается на меньшем объеме информации.
#  но если смотреть на метрики precision recall, то ситуация намного, намного лучше.
#  и если precision не сильно изменился, то recall вырос существенно. Т.е. мы лучше стали выделять нужный класс из общего массива
#  Ну я думаю это в любом случае связано с обрезанными данными, потому что один из самых простых способов борьбы с несбалансированными
# классами, это как раз обрезка классов....в данном случае мы впринципе это и делаем, не думаю что на модель как-то влияет
# кодировка на уровне 1/-1, но сам прием интересный)) возьму на вооружение

поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [63]:
# сделаем цикл

In [64]:
compare_p = pd.DataFrame(columns=["Threshold", "Precision", "Recall", "F1_score", "ROC_AUC"])
for p in np.arange(0.1, 1, 0.1):
    pu_train = prepare_df_for_pulearner(df, p, "target_class")
    pu_test = prepare_df_for_pulearner(df, p, "target_class")
    
    pu_X_train = pu_train.iloc[:, :-2]
    pu_y_train = pu_train.iloc[:, -1]
    pu_X_test = pu_test.iloc[:, :-2]
    pu_y_test = pu_test.iloc[:, -1]
    
    forest_pipe_for_PU.fit(pu_X_train, pu_y_train)
    forestPU_predict = forest_pipe_for_PU.predict(pu_X_test)
    
    roc_score = roc_auc_score(pu_y_test, forestPU_predict)
    
    precision, recall, thresholds = precision_recall_curve(pu_y_test, forestPU_predict)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    
    pu_model = pd.DataFrame(np.array([thresholds[ix], precision[ix], recall[ix], fscore[ix], roc_score]).reshape(1, -1), 
                                columns=["Threshold", "Precision", "Recall", "F1_score", "ROC_AUC"], 
                                index=[f"P_rate={p:%}"])
    compare_p = compare_p.append(pu_model)
    
    

Used 115 instances of 1153 total positive class
Used 115 instances of 1153 total positive class
Used 230 instances of 1153 total positive class
Used 230 instances of 1153 total positive class
Used 345 instances of 1153 total positive class
Used 345 instances of 1153 total positive class
Used 461 instances of 1153 total positive class
Used 461 instances of 1153 total positive class
Used 576 instances of 1153 total positive class
Used 576 instances of 1153 total positive class
Used 691 instances of 1153 total positive class
Used 691 instances of 1153 total positive class
Used 807 instances of 1153 total positive class
Used 807 instances of 1153 total positive class
Used 922 instances of 1153 total positive class
Used 922 instances of 1153 total positive class
Used 1037 instances of 1153 total positive class
Used 1037 instances of 1153 total positive class


In [65]:
compare_p

,Threshold,Precision,Recall,F1_score,ROC_AUC
P_rate=10.000000%,1.0,0.938053,0.921739,0.929825,0.930435
P_rate=20.000000%,1.0,0.963303,0.913043,0.937500,0.939130
P_rate=30.000000%,1.0,0.942943,0.910145,0.926254,0.927536
P_rate=40.000000%,1.0,0.960648,0.900217,0.929451,0.931670
P_rate=50.000000%,1.0,0.962433,0.934028,0.948018,0.948785
P_rate=60.000000%,1.0,0.974085,0.924747,0.948775,0.950072
P_rate=70.000000%,1.0,0.962581,0.924411,0.943110,0.944238
P_rate=80.000000%,1.0,0.964490,0.913232,0.938162,0.939805
P_rate=90.000000%,1.0,0.967677,0.923819,0.945239,0.946480


In [66]:
#  в целом думаю понятно что с ростом количества примеров растет и качество модели.....в основном конечно recall поскольку 
#  это основная мтерика на которую нужно обращать внимание при попытке выделить положительные классы